In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
# Charger les données prétraitées
X = pd.read_csv("data/X_matrix_preprocessed.csv")
Y = pd.read_csv("data/Y_matrix_preprocessed.csv")

In [ ]:
# Diviser les données en ensembles d'entraînement et de test
print("Division des données en ensembles d'entraînement et de test...")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

print(f"Dimensions de X_train : {X_train.shape}, X_test : {X_test.shape}")
print(f"Dimensions de y_train : {y_train.shape}, y_test : {y_test.shape}")

In [ ]:
# Définir une fonction pour entraîner ElasticNet
def train_elasticnet(X_train, y_train, X_test, y_test):
    """
    Entraîne un modèle ElasticNet et évalue ses performances.
    """
    # Définir les hyperparamètres à rechercher
    param_grid = {
        "alpha": [0.1, 0.5, 1.0, 5.0, 10.0],  # Paramètre de régularisation
        "l1_ratio": [0.1, 0.5, 0.7, 0.9, 1.0],  # Ratio entre L1 (LASSO) et L2 (Ridge)
    }

    # Initialiser ElasticNet avec un nombre d'itérations maximal élevé
    elastic_net = ElasticNet(max_iter=10000, random_state=42)

    # Utiliser GridSearchCV pour trouver les meilleurs hyperparamètres
    grid_search = GridSearchCV(estimator=elastic_net, param_grid=param_grid, cv=5, scoring="r2", verbose=2, n_jobs=-1)
    grid_search.fit(X_train, y_train.values.ravel())  # y_train.values.ravel() pour une sortie compatible avec ElasticNet

    # Meilleur modèle trouvé
    best_model = grid_search.best_estimator_

    # Prédictions sur les ensembles d'entraînement et de test
    y_train_pred = best_model.predict(X_train)
    y_test_pred = best_model.predict(X_test)

    # Évaluer les performances
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)

    print("\nMeilleurs hyperparamètres :", grid_search.best_params_)
    print("R² Score (train):", train_r2)
    print("R² Score (test):", test_r2)
    print("Mean Squared Error (train):", train_mse)
    print("Mean Squared Error (test):", test_mse)

    return best_model, y_train_pred, y_test_pred

In [ ]:
# Entraîner ElasticNet
print("Entraînement du modèle ElasticNet...")
best_model, y_train_pred, y_test_pred = train_elasticnet(X_train, y_train, X_test, y_test)

In [ ]:
# Visualiser les prédictions vs valeurs réelles
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, color="blue", label="Valeurs prédites")
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--", label="Valeurs réelles")
plt.xlabel("Valeurs réelles (y_test)")
plt.ylabel("Valeurs prédites (y_pred)")
plt.title("ElasticNet : Comparaison des valeurs réelles et prédites")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Afficher les coefficients importants
print("\nAffichage des coefficients importants...")
coefficients = pd.Series(best_model.coef_, index=X_train.columns)
coefficients_sorted = coefficients.sort_values(ascending=False)
top_features = coefficients_sorted.head(10)
print("\nTop 10 des coefficients les plus importants :")
print(top_features)

In [ ]:
# Visualiser les coefficients importants
plt.figure(figsize=(10, 6))
top_features.plot(kind="bar", color="orange")
plt.title("Top 10 des coefficients les plus importants (ElasticNet)")
plt.xlabel("Caractéristiques")
plt.ylabel("Valeur du coefficient")
plt.grid()
plt.show()